In [1]:
import argparse
import os
from pathlib import Path
import torch
import torch.nn as nn
from PIL import Image
from os.path import basename
from os.path import splitext
from torchvision import transforms
from torchvision.utils import save_image

%load_ext autoreload
%autoreload 2
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as T

import tensorflow.keras as keras
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

from operator import itemgetter
def print_public_attributes(obj):
    print(list([att for att in dir(obj) 
         if '__' not in att and not att.startswith('_')]))

In [2]:
def calc_mean_std_pt(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2] # N,C,H,W
    feat_var = feat.view(N, C, -1).var(dim=2) + eps
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std

def calc_mean_std_tf(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
#     feat_mean, feat_var = tf.nn.moments(feat,[2,3],keepdims=True,)
    shape = tf.shape(feat)
    bessel_correction = shape[2] * shape[3] / (shape[2] * shape[3] - 1)
    feat_flat = tf.reshape(feat,[shape[0],shape[1],shape[2] * shape[3]])
    feat_mean = tf.reduce_mean(feat_flat,[2],keepdims=True)
    feat_var = tf.math.reduce_variance(feat_flat,[2],keepdims=True) * bessel_correction + eps
    feat_std = tf.sqrt(feat_var)
#     feat_ += eps
#     feat_std = tf.sqrt(feat_var)
    return feat_mean, feat_std

with torch.no_grad():
    inval = np.arange(4).reshape(1,1,2,2) / 1.
    exp = calc_mean_std_pt(
        torch.tensor(inval)
    )
    act = calc_mean_std_tf(
        tf.constant(inval)
    )
    for a,b in zip(exp,act):
        assert np.allclose(a.numpy(),b.numpy())

2022-02-27 21:08:31.062191: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
tf.math.reduce_variance(inval), torch.var(torch.tensor(inval),unbiased=False)

(<tf.Tensor: shape=(), dtype=float64, numpy=1.25>,
 tensor(1.2500, dtype=torch.float64))

In [4]:
exp, act

((tensor([[[[1.5000]]]], dtype=torch.float64),
  tensor([[[[1.2910]]]], dtype=torch.float64)),
 (<tf.Tensor: shape=(1, 1, 1), dtype=float64, numpy=array([[[1.5]]])>,
  <tf.Tensor: shape=(1, 1, 1), dtype=float64, numpy=array([[[1.29099832]]])>))

In [5]:
def mean_variance_norm_pt(feat):
    size = feat.size()
    mean, std = calc_mean_std_pt(feat)
    normalized_feat = (feat - mean.expand(size)) / std.expand(size)
    return normalized_feat

def mean_variance_norm_tf(feat):
    shape = tf.shape(feat)
    mean, std = calc_mean_std_tf(feat)
    normalized_feat = (feat - tf.broadcast_to(mean,shape)) \
        / tf.broadcast_to(std,shape)
    return normalized_feat


with torch.no_grad():
    inval = np.arange(4).reshape(1,1,2,2) / 1.
    exp = mean_variance_norm_pt(
        torch.tensor(inval)
    )
    act = mean_variance_norm_tf(
        tf.constant(inval)
    )
    assert np.allclose(exp.numpy(),act.numpy())

In [6]:
calc_mean_std_tf(inval)

(<tf.Tensor: shape=(1, 1, 1), dtype=float64, numpy=array([[[1.5]]])>,
 <tf.Tensor: shape=(1, 1, 1), dtype=float64, numpy=array([[[1.29099832]]])>)

In [7]:
enc_arch = [
    
]

dec_arch = [
    ['refpad',[1,1,1,1]],
    ['conv',[512,256,(3,3)]],
    ['relu'],
    ['upsample',2],
    
    ['refpad',[1,1,1,1]],
    ['conv',[256,256,(3,3)]],
    ['relu'],
    ['refpad',[1,1,1,1]],
    ['conv',[256,256,(3,3)]],
    ['relu'],
    ['refpad',[1,1,1,1]],
    ['conv',[256,256,(3,3)]],
    ['relu'],
    ['refpad',[1,1,1,1]],
    ['conv',[256,128,(3,3)]],
    ['relu'],    
    ['upsample',2],
    
    
    ['refpad',[1,1,1,1]],
    ['conv',[128,128,(3,3)]],
    ['relu'],    
    ['refpad',[1,1,1,1]],
    ['conv',[128,64,(3,3)]],
    ['relu'],    
    ['upsample',2],
    
    ['refpad',[1,1,1,1]],
    ['conv',[64,64,(3,3)]],
    ['relu'],    
    ['refpad',[1,1,1,1]],
    ['conv',[64,3,(3,3)]],
]

In [8]:
x = torch.tensor(inval)